This file runs all the models developed in this work. The operational models are listed after the defined functions and classes. Please note that the file paths are configured to match my environment. To use the models for making predictions, you will need to update these paths to match your own setup.

## Defines all Necessary Functions and Classes

In [1]:
# Imports all the necessary packages
import pandas as pd
import rdkit
import rdkit.Chem
import numpy as np
import random
import os
import torch
import torch.nn as nn
from torch.utils.data import random_split
from torch_geometric.nn import global_add_pool, GCNConv
from torch.utils.data import DataLoader, Dataset
import pandas as pd

pd.set_option('display.max_columns', None)

CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

# My data only consists of 9 different elements. These are given in the dictionary below, along with their atom number.
elements_allowed = {1: "H", 6: "C", 7: "N", 8: "O", 9: "F", 16: "S", 17: "Cl", 35: "Br", 53: "I"}
num_of_features = 9
best_config = {'num_gcn_layers': 4, 'num_hidden_layers': 4, 'GCN_output_per_layer': [100, 420, 140, 140], 'hidden_neurons_per_layer': [260, 60, 180, 100], 'learning_rate': 0.007425096728429009, 'size_of_batch': 128, 'number_of_epochs': 600, 'number_of_temp': 7, 'min_interval': 20, 'patience': 50, 'dropout_rate': 0.2,  'activation_function': 'relu', 'decay_rate': 0.95}



def extract_training_SMILES_temperature_and_target(excel_filename, path_to_code):
    """
    Extracts the data that will be used for training and validating the model. This data is found in a folder called
    "T=temperature", where temperature is the temperature we want to focus on. The data is found in a file called
    "Development.xlsx" in this folder, which contains the SMILES notation for all the compounds, along with the
    corresponding vapor pressure at the given temperature. Will by default extract the data from the file
    "Development.xlsx".
    """

    # Locates the file for training data.
    excel_data = pd.read_excel(f"{path_to_code}/Data/{excel_filename}")

    X_dev = []
    y_dev = []
    temp = []    

    for _, row in excel_data.iterrows():
        SMILES = row['SMILES']
        for column in row.index:
            if column == 'SMILES':
                continue
            
            if column.startswith('T') and not column.startswith('TMIN') and not column.startswith('TMAX') and not column.startswith('Temperature Interval'):
                
                if not pd.isna(row[column]):  # Check if the value is not NaN
                    # Extracts the temperature and the logP value from the cells. 
                    cell_value = row[column].split("=")
                    temperature = float(cell_value[0][2:-2])
                    logP = float(cell_value[1])

                    # Appends the values to the lists.
                    X_dev.append(SMILES)
                    y_dev.append(logP)
                    temp.append(float(temperature))

    return X_dev, y_dev, temp


def extract_mean_std(number_of_temp, min_interval):
    """
    Extracts the mean and standard deviation for the training data. 
    """
    with open(f"Data/Min interval of {min_interval} ºC/{number_of_temp} temperatures/Pressure Overview (log).txt", 'r') as file:
        data = file.readlines()
        for line in data: 
            if line.startswith("Mean pressure in training data"):
                mean = float(line.split(":")[1].strip())
            if line.startswith("Standard deviation of pressure in training data"):
                std = float(line.split(":")[1].strip())

    return mean, std


def shuffle_data(X_dev, y_dev, temp):
    """
    This function shuffles the data. 
    """

    # We have to shuffle the temperature and target values in the same way as the SMILES values.
    shuffled_indices = list(range(len(X_dev)))
    random.shuffle(shuffled_indices)

    X_dev = [X_dev[i] for i in shuffled_indices]
    y_dev = [y_dev[i] for i in shuffled_indices]
    temp = [temp[i] for i in shuffled_indices]

    return X_dev, y_dev, temp


def set_seed(seed_value):
    """Set seed for reproducibility."""
    np.random.seed(seed_value)  # Set numpy seed
    torch.manual_seed(seed_value)  # Set torch seed
    random.seed(seed_value)  # Set python random seed
    os.environ['PYTHONHASHSEED'] = str(seed_value)  # Set python environment seed


def smiles2graph(sml):
    """
    This code is based on the code from the book "Deep Learning for Molecules and Materials" byAndrew D White.
    This function will return the graph of a molecule based on the SMILES string!
    """
    m = rdkit.Chem.MolFromSmiles(sml)
    m = rdkit.Chem.AddHs(m)
    order_string = {
        rdkit.Chem.rdchem.BondType.SINGLE: 1,
        rdkit.Chem.rdchem.BondType.DOUBLE: 2,
        rdkit.Chem.rdchem.BondType.TRIPLE: 3,
        rdkit.Chem.rdchem.BondType.AROMATIC: 4,
    }

    # The length of the adjacency matrix should be NxN, where N is the number of atoms in the molecule.
    N = len(list(m.GetAtoms()))
    nodes = np.zeros((N, len(elements_allowed)))
    lookup = list(elements_allowed.keys())
    for i in m.GetAtoms():
        # If an atom is present in our molecule,
        nodes[i.GetIdx(), lookup.index(i.GetAtomicNum())] = 1

    adj = np.zeros((N, N))
    for j in m.GetBonds():
        u = min(j.GetBeginAtomIdx(), j.GetEndAtomIdx())
        v = max(j.GetBeginAtomIdx(), j.GetEndAtomIdx())
        order = j.GetBondType()
        if order in order_string:
            order = order_string[order]
        else:
            raise Warning("Ignoring bond order" + order)
        adj[u, v] = 1
        adj[v, u] = 1
    # We want the diagonal in the matrix to be 1.
    adj += np.eye(N)
    return nodes, adj


class MolecularDataset(Dataset):
    """
    This class is needed to create our dataset (on the Dataset format).
    The class inherits from the Dataset class.
    """

    def __init__(self, X, y, temperature):
        # Initializes the features and targets. Is out constructor.
        self.X = X
        self.y = y
        self.temperature = temperature

    def __len__(self):
        # Returns the length of the dataset
        return len(self.X)

    def __getitem__(self, idx):
        # Extract the SMILES value of the molecule, calculate the graphs based on that SMILE, and then return this
        # value along with the corresponding target value.
        SMILES = self.X[idx]
        nodes, adj = smiles2graph(SMILES)

        # Convert nodes and adj to tensors, assuming they are NumPy arrays returned from smiles2graph
        nodes_tensor = torch.tensor(nodes, dtype=torch.float32)
        adj_tensor = torch.tensor(adj, dtype=torch.float32)

        # Convert the target value to a tensor. Assuming solubility is a single floating-point value.
        target_tensor = torch.tensor(self.y[idx], dtype=torch.float32)

        # Convert the temperature to a tensor. Assuming temperature is a single floating-point value.
        temperature_tensor = torch.tensor(self.temperature[idx], dtype=torch.float32)

        return (nodes_tensor, adj_tensor), target_tensor, temperature_tensor


def split_dataset(dataset, test_split=0.2):
    """
    Splits the dataset into training and testing datasets. Will split the dataset into 80% training and 20% testing
    if no test_split is specified.
    """
    # Determine the lengths
    test_size = int(test_split * len(dataset))
    train_size = len(dataset) - test_size

    # Sets a seed for reproducibility
    generator = torch.Generator().manual_seed(42)

    # Split the dataset. Implements a seed for reproducibility.
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=generator)

    return train_dataset, test_dataset


def merge_batch(batch):
    """
    This function merge the graphs in the batch into a larger batch by concatenating node features and computing new
    edge indices. Batch is on the format [(graph1, target1), (graph2, target2), ...], and the output is on the format
    (merged_nodes, merged_edge_indices, merged_batch_mapping), merged_targets, merged_temperatures.

    Further, merged_nodes is on the format [node1, node2, ...], merged_edge_indices is on the format [edge1, edge2, ...]
    and merged_batch_mapping is on the format [0, 0, ..., 1, 1, ...], where the length of the lists are the number of
    nodes and edges in the merged graph, respectively. The merged_targets is on the format [target1, target2, ...].
    The merged_temperatures is on the format [temperature1, temperature2, ...].
    """
    # Separate out nodes, adjacency matrices, scalar tensors, and temperatures
    nodes_list = [item[0][0] for item in batch]
    adj_list = [item[0][1] for item in batch]
    scalar_tensors = [item[1] for item in batch]
    temperatures = [item[2] for item in batch]

    # Placeholder for the combined nodes
    merged_nodes = []
    # Placeholder for the combined edge indices (plural for index) -> says something about connections
    edge_indices = []
    # Placeholder for the batch mapping -> says something about which node that belongs to which graph! Important for
    # tying things together.
    batch_mapping = []

    # This will keep track of the current node index offset in the combined graph -> how much we must "shift" the
    # edge matrix
    current_node_index = 0

    # Iterates over all the graphs
    for idx, (nodes, adj) in enumerate(zip(nodes_list, adj_list)):
        # Extracts the number of nodes in the current graph.
        num_nodes = nodes.shape[0]

        # Add the current graph's nodes to the merged node list. Can append as usual, since we want to add rows.
        merged_nodes.append(nodes)

        # Converts the adjacency matrix to the correct edge index format required for the GCN layer. In other words, we
        # find the edges in the adjacency matrix, and offset them by the current node index.
        edges = adj.nonzero().t()
        edges = edges + current_node_index

        # Add the current graph's edges to the combined list
        edge_indices.append(edges)

        # Create the batch mapping for the current graph
        batch_mapping.extend([idx] * num_nodes)

        # Update the node index offset
        current_node_index += num_nodes

    # Convert the merged node list to a tensor
    merged_nodes_tensor = torch.cat(merged_nodes, dim=0)

    # Convert the edge index lists to a single tensor
    merged_edge_indices_tensor = torch.cat(edge_indices, dim=1)

    # Convert the batch mapping to a tensor
    batch_mapping_tensor = torch.tensor(batch_mapping, dtype=torch.long)

    # Convert scalar_tensors (target values) into a single tensor (can be converted directly).
    merged_scalar_tensor = torch.stack(scalar_tensors)

    # Convert temperatures into a single tensor.
    merged_temperature_tensor = torch.stack(temperatures)

    return (merged_nodes_tensor, merged_edge_indices_tensor, batch_mapping_tensor), merged_scalar_tensor, merged_temperature_tensor


class AntoineEquationLayer(nn.Module):
    def __init__(self):
        """
        Initialize the custom layer. 
        """
        super(AntoineEquationLayer, self).__init__()

    def forward(self, x_in, temperature):
        """
        Forward pass of the layer using the Antoine equation.
        x_in is expected to have three elements representing A, B, and C coefficients. Note that x_in will have the
        shape (batch_size, 3). This layer also expect a temperature as input.
        """
    
        A, B, C = x_in[:, 0], x_in[:, 1], x_in[:, 2]

        # Here, P will be the pressure in mmHg! 
        log_P = A - B / (temperature + C)

        return log_P, A, B, C


class GNNModel(nn.Module):
    """
    This class defines the structure of the model. The model will be a graph neural network (GNN) model. The model
    inherits from the nn.Module class, which is the base class for all neural network modules in PyTorch. 
    """

    def __init__(self, num_of_features, num_gcn_layers, num_hidden_layers, num_dense_neurons, GCN_output_per_layer, 
                 dropout_rate, activation_function):
        # Defines the structure of the model. 
        super().__init__()

        # Set the activation function.
        if activation_function == "relu":
            self.activation = nn.ReLU()

        elif activation_function == "sigmoid":
            self.activation = nn.Sigmoid()

        elif activation_function == "tanh":
            self.activation = nn.Tanh()

        # Initialize GCN layers and activations as ModuleList
        self.GCN_layers = nn.ModuleList()
        self.GCN_activations = nn.ModuleList()

        # Adds the GCN layers and the activation functions to the model.
        for i in range(num_gcn_layers):
            self.GCN_layers.append(GCNConv(num_of_features, GCN_output_per_layer[i]))
            self.GCN_activations.append(self.activation)
            num_of_features = GCN_output_per_layer[i]

        # Adds the global pooling layer.
        self.global_pooling = global_add_pool
        self.global_pooling_activation = self.activation

        # Initialize dense layers and activations as ModuleList
        self.dense_layers = nn.ModuleList()
        self.dense_activations = nn.ModuleList()
        self.dropout = nn.ModuleList()

        # Adds the dense layers and the activation functions to the model.
        for i in range(num_hidden_layers):
            self.dense_layers.append(nn.Linear(num_of_features, num_dense_neurons[i]))
            self.dense_activations.append(self.activation)
            self.dropout.append(nn.Dropout(p=dropout_rate))
            num_of_features = num_dense_neurons[i]

        # Adds the Antoine coefficients layer.
        self.antonine_coeff = nn.Linear(num_of_features, 3)
        
        # Create the Antinone equation layer separatly as it is not possible to pass additional arguments to the
        # Sequential class. However, we need to pass the temperature to the Antoine equation layer.
        self.antonine_layer = AntoineEquationLayer()

    def forward(self, x, edge_indices, batch_mapping, temperature, mean, std):
        # Defines the forward pass of the model. This is where the data is input to the model.

        #print(f"Input shape: {x.shape}")
        # Iterates over all the GCN layers and the activation functions.
        for layer, act in zip(self.GCN_layers, self.GCN_activations):
            # Performs the message passing and then applies the activation function. Edge index says which atoms 
            # that are connected. 
            x = act(layer(x, edge_indices))
            #print(f"After GCN Layer {layer}: {x.shape}")

        # Apply global pooling. Here we need batch_mapping to map which atoms that belongs to which molecule.
        x = self.global_pooling(x, batch_mapping)
        x = self.global_pooling_activation(x)
        #print(f"After Global Pooling: {x.shape}")

        # Iterates over all the dense layers and the activation functions.
        for layer, act, drop in zip(self.dense_layers, self.dense_activations, self.dropout):
            # Applies the dense layer and then the activation function.
            x = act(layer(x))
            # Apply dropout
            x = drop(x)
            #print(f"After Dense Layer {layer}: {x.shape}")

        # Apply the Antoine coefficients layer
        x = self.antonine_coeff(x)
        #print(f"After Antoine Coefficients Layer: {x.shape}")

        log_P, A, B, C = self.antonine_layer(x, temperature)
        #print(f"Final Output Shape: {log_P.shape}")

        # Scale the output
        log_P = (log_P - mean) / std

        # Note that here I also return the Antoine coefficients. This was not done when the model was trained, but want to do it now to see 
        # the values.
        return log_P, A, B, C


def extract_VLE_data(excel_filename, path_to_code):
    """
    Extracts the data that will be used for training and validating the model.
    """

    # Locates the file for the training data.
    excel_file_VLE = pd.ExcelFile(f"{path_to_code}/Data/Combined model/{excel_filename}")

    # Locates the file for the SMILES notation.
    smiles_converter = pd.read_excel(f"{path_to_code}/Data/All_amines.xlsx")

    # Only want the columns "CAS No" and "SMILES"
    smiles_converter = smiles_converter[["CAS No", "SMILES"]]

    # Dictionary to store the data. The key is the SMILES notation and the value is the VLE data.
    dev_dict = {}

    for sheet in excel_file_VLE.sheet_names:
        # Get the correct SMILES notation for the compound.
        smiles_value = smiles_converter.loc[smiles_converter["CAS No"] == sheet, "SMILES"].values[0]
        dev_dict[smiles_value] = pd.read_excel(excel_file_VLE, sheet_name=sheet)

    return dev_dict


def saturation_pressure_water(T):
    """
    This function calculates the saturation pressure of water at a given temperature. These formulas are based on the
    Antoine equation recived from the Dortmund Data Bank.
    """

    # Initialize a tensor for P with the same shape as T and fill it with NaNs.
    P = torch.full_like(T, float('nan'))

    # Condition for temperature range 1 to 100 degrees Celsius
    condition1 = (T > 1) & (T < 100)
    A1 = 8.07131
    B1 = 1730.63
    C1 = 233.426
    P[condition1] = 10**(A1 - B1 / (T[condition1] + C1))

    # Condition for temperature range 100 to 374 degrees Celsius
    condition2 = (T >= 100) & (T < 374)
    A2 = 8.14019
    B2 = 1810.94
    C2 = 244.485
    P[condition2] = 10**(A2 - B2 / (T[condition2] + C2))

    return P


class NRTLEquationLayer(nn.Module):
    def __init__(self):
        """
        Initialize the custom layer. 
        """
        super(NRTLEquationLayer, self).__init__()

    def forward(self, x_in, temperature, mole_frac_amine, mole_frac_water):
        """
        x_in have four elements representing alpha, b_12, and b_21 coefficients.
        """

        # Extract the interaction parameters. 
        alpha, b_12, b_21 = x_in[:, 0], x_in[:, 1], x_in[:, 2]

        # Sets alpha to be constant at 0.3 -> Better to let it vary. 
        #alpha = 0.3
        alpha_min = 0.2
        alpha_max = 0.47

        # Scales alpha from 0 to 1
        #alpha = torch.sigmoid(alpha)
        # Scales alpha from [alpha_min, alpha_max]
        #alpha = alpha_min + (alpha_max - alpha_min) * alpha

        # Calculate alpha to get it in that range (from spt-nrtl paper)
        alpha = alpha_min * (1 + torch.sigmoid(alpha)/10 * ((alpha_max/alpha_min)))

        # Gas constant in mmHg
        R = 62.36367 # mmHg L / mol K

        # The temperature in Kelvin
        temperature = temperature + 273.15

        # Calculate tau 
        tau_12 = b_12/(R*temperature)
        tau_21 = b_21/(R*temperature)

        # Calculate G
        G_12 = torch.exp(-tau_12 * alpha)
        G_21 = torch.exp(-tau_21 * alpha)

        # Calculate the activity coefficients
        ln_gamma_amine = mole_frac_water**2 * (tau_21 * (G_21 / (mole_frac_amine + mole_frac_water * G_21) )**2 + (tau_12 * G_12)/( mole_frac_water + mole_frac_amine*G_12 )**2)
        ln_gamma_water = mole_frac_amine**2 * (tau_12 * (G_12 / (mole_frac_water + mole_frac_amine * G_12) )**2 + (tau_21 * G_21)/( mole_frac_amine + mole_frac_water*G_21 )**2)

        return ln_gamma_amine, ln_gamma_water


class GNN_Activity_coeff(nn.Module):
    """
    This class defines the structure of the model. The model will be a graph neural network (GNN) model. The model
    inherits from the nn.Module class, which is the base class for all neural network modules in PyTorch. 
    """

    def __init__(self, config, num_of_features):
        # Defines the structure of the model. 
        super().__init__()

        # Set the activation function.
        if config["activation_function"] == "relu":
            self.activation = nn.ReLU()

        elif config["activation_function"] == "sigmoid":
            self.activation = nn.Sigmoid()

        elif config["activation_function"] == "tanh":
            self.activation = nn.Tanh()

        # Initialize GCN layers and activations as ModuleList
        self.GCN_layers = nn.ModuleList()
        self.GCN_activations = nn.ModuleList()

        # Adds the GCN layers and the activation functions to the model.
        for i in range(best_config["num_gcn_layers"]):
            self.GCN_layers.append(GCNConv(num_of_features, best_config["GCN_output_per_layer"][i]))
            self.GCN_activations.append(self.activation)
            num_of_features = best_config["GCN_output_per_layer"][i]

        # Adds the global pooling layer.
        self.global_pooling = global_add_pool
        self.global_pooling_activation = self.activation

        # Initialize dense layers and activations as ModuleList
        self.dense_layers = nn.ModuleList()
        self.dense_activations = nn.ModuleList()
        self.dropout = nn.ModuleList()

        # Adds the dense layers and the activation functions to the model.
        for i in range(best_config["num_hidden_layers"]):
            self.dense_layers.append(nn.Linear(num_of_features, best_config["hidden_neurons_per_layer"][i]))
            self.dense_activations.append(self.activation)
            self.dropout.append(nn.Dropout(p=best_config["dropout_rate"]))
            num_of_features = best_config["hidden_neurons_per_layer"][i]

        # Adds the NRTL coefficients layer.
        self.NRTL_coeff = nn.Linear(num_of_features, 3)
        
        # Initilize the NRTRL layer
        self.nrtl_layer = NRTLEquationLayer()

    def forward(self, x, edge_indices, batch_mapping, temperature, mean, std, mole_frac_amine, mole_frac_water):
        # Defines the forward pass of the model. This is where the data is input to the model.

        #print(f"Input shape: {x.shape}")
        # Iterates over all the GCN layers and the activation functions.
        for layer, act in zip(self.GCN_layers, self.GCN_activations):
            # Performs the message passing and then applies the activation function. Edge index says which atoms 
            # that are connected. 
            x = act(layer(x, edge_indices))
            #print(f"After GCN Layer {layer}: {x.shape}")

        # Apply global pooling. Here we need batch_mapping to map which atoms that belongs to which molecule.
        x = self.global_pooling(x, batch_mapping)
        x = self.global_pooling_activation(x)
        #print(f"After Global Pooling: {x.shape}")

        # Iterates over all the dense layers and the activation functions.
        for layer, act, drop in zip(self.dense_layers, self.dense_activations, self.dropout):
            # Applies the dense layer and then the activation function.
            x = act(layer(x))
            # Apply dropout
            x = drop(x)
            #print(f"After Dense Layer {layer}: {x.shape}")

        # Apply the Antoine coefficients layer
        x = self.NRTL_coeff(x)
        #print(f"After NRTL Coefficients Layer: {x.shape}")

        ln_gamma_amine, ln_gamma_water = self.nrtl_layer(x, temperature, mole_frac_amine, mole_frac_water)
        #print(f"Final Output Shape: {log_P.shape}")

        return ln_gamma_amine, ln_gamma_water


class VLE_model(nn.Module):
    """
    This class defines the structure of the model. The model will be a graph neural network (GNN) model. The model
    inherits from the nn.Module class, which is the base class for all neural network modules in PyTorch. 
    """

    def __init__(self, GCN_saturation, mean_gcn, std_gcn, best_config, num_of_features):
        # Defines the structure of the model. 
        super().__init__()

        # Initialize the GCN saturation model with the pre-trained model. Also initilize the mean and standard deviation for scaling.
        self.GCN_saturation = GCN_saturation
        self.mean_gcn = mean_gcn
        self.std_gcn = std_gcn

        # Initilize the NRTL model
        self.NRTL_model = GNN_Activity_coeff(best_config, num_of_features)


    def forward(self, batch):
        """ Defines the forward pass of the model. This is where the data is input to the model. """

        # Unpack the batch
        (x, edge_indices, batch_mapping), x_water, x_amine, y_water, y_amine, temperature, P_tot = batch

        # Calculate the predictions of the GCN model
        log_P_normalized = self.GCN_saturation(x, edge_indices, batch_mapping, temperature, self.mean_gcn, self.std_gcn)
        # Only the first element as the others are the antoine coefficents
        log_P_normalized = log_P_normalized[0]
        log_P = log_P_normalized * self.std_gcn + self.mean_gcn
        P_amine = 10**log_P
        ln_P_amine = torch.log(P_amine)

        # Convert total pressure from Pa to mmHg
        P_tot = P_tot / 133.322

        # Take the natural logarithm of the total pressure, water fraction and amine fraction
        ln_P_tot = torch.log(P_tot)
        ln_x_water = torch.log(x_water)
        ln_x_amine = torch.log(x_amine)

        # Calculate the predictions of the NRTL model (activity coefficients)
        ln_gamma_amine, ln_gamma_water = self.NRTL_model(x, edge_indices, batch_mapping, temperature, self.mean_gcn, self.std_gcn, x_amine, x_water)

        # Calculate the saturation pressure of water
        P_water = saturation_pressure_water(temperature)
        ln_P_water = torch.log(P_water)

        # Calculate ln_y_amine and ln_y_water
        ln_y_amine_pred = ln_x_amine + ln_gamma_amine + ln_P_amine - ln_P_tot
        ln_y_water_pred = ln_x_water + ln_gamma_water + ln_P_water - ln_P_tot

        # Want to return the predictions of the amine and water fractions. 
        sum_ln_y = ln_y_amine_pred + ln_y_water_pred

        return torch.stack((ln_y_amine_pred, ln_y_water_pred, sum_ln_y), dim=1)


def merge_batch_VLE(batch):
    """
    This function merge the graphs in the batch into a larger batch by concatenating node features and computing new
    edge indices. Batch is on the format [(graph1, target1), (graph2, target2), ...], and the output is on the format
    (merged_nodes, merged_edge_indices, merged_batch_mapping), merged_targets, merged_temperatures.

    Further, merged_nodes is on the format [node1, node2, ...], merged_edge_indices is on the format [edge1, edge2, ...]
    and merged_batch_mapping is on the format [0, 0, ..., 1, 1, ...], where the length of the lists are the number of
    nodes and edges in the merged graph, respectively. The merged_targets is on the format [target1, target2, ...].
    The merged_temperatures is on the format [temperature1, temperature2, ...].
    """

    # Separate out nodes, adjacency matrices, scalar tensors, and temperatures
    nodes_list = [item[0][0] for item in batch]
    adj_list = [item[0][1] for item in batch]
    x_water_tensor = [item[1] for item in batch]
    x_amine_tensor = [item[2] for item in batch]
    y_water_tensor = [item[3] for item in batch]
    y_amine_tensor = [item[4] for item in batch]
    temperatures = [item[5] for item in batch]
    P_tot = [item[6] for item in batch]

    # Placeholder for the combined nodes
    merged_nodes = []
    # Placeholder for the combined edge indices (plural for index) -> says something about connections
    edge_indices = []
    # Placeholder for the batch mapping -> says something about which node that belongs to which graph! Important for
    # tying things together.
    batch_mapping = []

    # This will keep track of the current node index offset in the combined graph -> how much we must "shift" the
    # edge matrix
    current_node_index = 0

    # Iterates over all the graphs
    for idx, (nodes, adj) in enumerate(zip(nodes_list, adj_list)):
        # Extracts the number of nodes in the current graph.
        num_nodes = nodes.shape[0]

        # Add the current graph's nodes to the merged node list. Can append as usual, since we want to add rows.
        merged_nodes.append(nodes)

        # Converts the adjacency matrix to the correct edge index format required for the GCN layer. In other words, we
        # find the edges in the adjacency matrix, and offset them by the current node index.
        edges = adj.nonzero().t()
        edges = edges + current_node_index

        # Add the current graph's edges to the combined list
        edge_indices.append(edges)

        # Create the batch mapping for the current graph
        batch_mapping.extend([idx] * num_nodes)

        # Update the node index offset
        current_node_index += num_nodes

    # Convert the merged node list to a tensor
    merged_nodes_tensor = torch.cat(merged_nodes, dim=0)

    # Convert the edge index lists to a single tensor
    merged_edge_indices_tensor = torch.cat(edge_indices, dim=1)

    # Convert the batch mapping to a tensor
    batch_mapping_tensor = torch.tensor(batch_mapping, dtype=torch.long)

    # Convert x_water_tensor, x_amine_tensor, y_water_tensor, y_amine_tensor, temperature_tensor and P_tot_tensor to single tensors.
    x_water_tensor = torch.stack(x_water_tensor)
    x_amine_tensor = torch.stack(x_amine_tensor)
    y_water_tensor = torch.stack(y_water_tensor)
    y_amine_tensor = torch.stack(y_amine_tensor)
    temperature_tensor = torch.stack(temperatures)
    P_tot_tensor = torch.stack(P_tot)

    return (merged_nodes_tensor, merged_edge_indices_tensor, batch_mapping_tensor), x_water_tensor, x_amine_tensor, y_water_tensor, y_amine_tensor, temperature_tensor, P_tot_tensor


class MolecularDatasetVLE(Dataset):
    """
    This class is needed to create our dataset (on the Dataset format).
    The class inherits from the Dataset class. Input is on format X and T, where X is the SMILES notation and T is the
    temperature. 
    """

    def __init__(self, data):
        # Initializes the features and targets. Is our constructor.
        self.SMILES = data[0]
        self.x_water = data[1]
        self.x_amine = data[2]
        self.y_water = data[3]
        self.y_amine = data[4]
        self.temperature = data[5]
        self.P_tot = data[6]

    def __len__(self):
        # Returns the length of the dataset
        return len(self.SMILES)

    def __getitem__(self, idx):
        # Extract the SMILES value of the molecule, calculate the graphs based on that SMILE, and then return this
        # value along with the corresponding target value.
        SMILES_one_molecule = self.SMILES[idx]
        nodes, adj = smiles2graph(SMILES_one_molecule)

        # Convert nodes and adj to tensors, assuming they are NumPy arrays returned from smiles2graph
        nodes_tensor = torch.tensor(nodes, dtype=torch.float32)
        adj_tensor = torch.tensor(adj, dtype=torch.float32)

        # Convert x_water, x_amine, y_water, y_amine, temperature and pressure to tensors.
        x_water_tensor = torch.tensor(self.x_water[idx], dtype=torch.float32)
        x_amine_tensor = torch.tensor(self.x_amine[idx], dtype=torch.float32)
        y_water_tensor = torch.tensor(self.y_water[idx], dtype=torch.float32)
        y_amine_tensor = torch.tensor(self.y_amine[idx], dtype=torch.float32)
        temperature_tensor = torch.tensor(self.temperature[idx], dtype=torch.float32)
        P_tot_tensor = torch.tensor(self.P_tot[idx], dtype=torch.float32)

        return (nodes_tensor, adj_tensor), x_water_tensor, x_amine_tensor, y_water_tensor, y_amine_tensor, temperature_tensor, P_tot_tensor

/Users/kristoffer/anaconda3/envs/master_thesis_environment_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-05 12:45:24,424	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-05 12:45:24,565	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## General Saturation Pressure Model

In [2]:
def model_prediction_general(SMILES, temperature):

    # Initilize the model with the best hyperparameters.
    model = GNNModel(num_of_features, best_config["num_gcn_layers"], best_config["num_hidden_layers"], best_config["hidden_neurons_per_layer"], best_config["GCN_output_per_layer"], best_config["dropout_rate"], best_config["activation_function"])

    # Load the best model.
    state_dict = torch.load(f"Data/Best general model/Best_model.pt")
    model.load_state_dict(state_dict)
    model.eval() 

    # Create a dummy dataset for the y_values (will only be used for converting the SMILES etc to correct format for model)
    y_dev = [1]*len(SMILES)

    # Need to extract the mean and standard deviation for scaling. 
    mean, std = extract_mean_std(best_config["number_of_temp"], best_config["min_interval"])

    # Convert dataset to graphs etc. 
    converted_dataset = MolecularDataset(SMILES, y_dev, temperature)

    # Creating a DataLoader. 
    loader = DataLoader(converted_dataset, batch_size=1, collate_fn=merge_batch, shuffle=False)

    with torch.no_grad(): 
        prediceted_pressure_values = []
        A_list = []
        B_list = []
        C_list = []
        for batch in loader:

            (nodes, edge_indices, batch_mapping), targets, temperature = batch

            # Calculate the predictions. 
            y_hat, A, B, C = model(nodes, edge_indices, batch_mapping, temperature, mean, std)

            # Scale the output back to the original scale.
            y_hat = y_hat * std + mean
            
            # Add the predicted value to the list.
            prediceted_pressure_values.append(y_hat.item())
            A_list.append(A.item())
            B_list.append(B.item())
            C_list.append(C.item())

    return prediceted_pressure_values, A_list, B_list, C_list

In [8]:
SMILES = "C#CCCC#C"
temperature = 40             # Should be given in C

saturation_pressure, A_pred, B_pred, C_pred = model_prediction_general([SMILES], [temperature])

print(f"The saturation pressure of {SMILES} at {temperature} C is {saturation_pressure[0]} mmHg")
print(f"The Antoine coefficients are A = {A_pred[0]}, B = {B_pred[0]} and C = {C_pred[0]}")

The saturation pressure of C#CCCC#C at 40 C is 2.174471378326416 mmHg
The Antoine coefficients are A = 3.6643755435943604, B = 76.98932647705078 and C = 11.674010276794434


## Amine Saturation Pressure Model

In [3]:
def model_prediction_amine(SMILES, temperature):

    model = GNNModel(num_of_features, best_config["num_gcn_layers"], best_config["num_hidden_layers"], best_config["hidden_neurons_per_layer"], best_config["GCN_output_per_layer"], best_config["dropout_rate"], best_config["activation_function"])

    # Load the best model.
    state_dict = torch.load(f"Data/Best amine model/Best_model.pt")
    model.load_state_dict(state_dict)
    model.eval() 

    # Create a dummy dataset for the y_values (will only be used for converting the SMILES etc to correct format for model)
    y_dev = [1]*len(SMILES)

    # Need to extract the mean and standard deviation for scaling. 
    mean, std = extract_mean_std(best_config["number_of_temp"], best_config["min_interval"])

    # Convert dataset to graphs etc. 
    converted_dataset = MolecularDataset(SMILES, y_dev, temperature)

    # Creating a DataLoader. 
    loader = DataLoader(converted_dataset, batch_size=1, collate_fn=merge_batch, shuffle=False)

    with torch.no_grad(): 
        prediceted_pressure_values = []
        A_list = []
        B_list = []
        C_list = []
        for batch in loader:

            (nodes, edge_indices, batch_mapping), targets, temperature = batch

            # Calculate the predictions. 
            y_hat, A, B, C = model(nodes, edge_indices, batch_mapping, temperature, mean, std)

            # Scale the output back to the original scale.
            y_hat = y_hat * std + mean
            
            # Add the predicted value to the list.
            prediceted_pressure_values.append(y_hat.item())
            A_list.append(A.item())
            B_list.append(B.item())
            C_list.append(C.item())

    return prediceted_pressure_values, A_list, B_list, C_list

In [9]:
SMILES = "C(CCN)CN"
temperature = 40             # Should be given in C

saturation_pressure, A_pred, B_pred, C_pred = model_prediction_amine([SMILES], [temperature])

print(f"The saturation pressure of {SMILES} at {temperature} C is {saturation_pressure[0]} mmHg")
print(f"The Antoine coefficients are A = {A_pred[0]}, B = {B_pred[0]} and C = {C_pred[0]}")

The saturation pressure of C(CCN)CN at 40 C is 0.6853728294372559 mmHg
The Antoine coefficients are A = 3.964467763900757, B = 282.623291015625 and C = 46.18942642211914


## VLE Model

In [5]:
def VLE_prediction(SMILES, temperature, x_water, x_amine, P_tot):

    # Extracts the mean and standard deviation for scaling. We use the same scaling as before. 
    mean_gcn, std_gcn = extract_mean_std(best_config["number_of_temp"], best_config["min_interval"])

    # Create a dummy dataset for the y_values (will only be used for converting the SMILES etc to correct format for model)
    y_water = [1]*len(SMILES)
    y_amine = [1]*len(SMILES)
    data = [[SMILES], [x_water], [x_amine], y_water, y_amine, [temperature], [P_tot]]

    # Convert SMILES to graphs etc.
    converted_dataset = MolecularDatasetVLE(data)

    # Creating a DataLoader.
    loader = DataLoader(converted_dataset, batch_size=1, collate_fn=merge_batch_VLE, shuffle=False)

    # Initilize the model with the best hyperparameters. # Initilize and load the best gcn model for predicting amine pressure from earlier training.
    gcn_saturation = GNNModel(num_of_features, best_config["num_gcn_layers"], best_config["num_hidden_layers"], best_config["hidden_neurons_per_layer"], best_config["GCN_output_per_layer"], best_config["dropout_rate"], best_config["activation_function"])

    # Initilize the new model that will be trained.
    model = VLE_model(gcn_saturation, mean_gcn, std_gcn, best_config, num_of_features)

    # Now want to evaluate how this model performs on the training data. 
    # Load the already existing model 
    model.load_state_dict(torch.load(f"Data/Best combined model/Best_model.pt"))

    # Put the model in evaluation mode
    model.eval()
    
    with torch.no_grad():
        for index, batch in enumerate(loader):
            # Calculate the predictions for the given batch.
            y_hat = model(batch)

            return y_hat

In [4]:
SMILES = "CN1CCNCC1"
x_water = 0.5414
x_amine = 1 - x_water
temperature = 115.05             # Should be given in C
P_tot = 101300                   # Should be given in Pa
prediction = VLE_prediction(SMILES, temperature, x_water, x_amine, P_tot)

ln_y_amine, ln_y_water = prediction[0][0].item(), prediction[0][1].item()

print("ln_y_amine: ", ln_y_amine)
print("ln_y_water: ", ln_y_water)

ln_y_amine:  -1.7621064186096191
ln_y_water:  -0.12092828750610352
